In [15]:
import numpy as np
import itertools

In [82]:
def kernelize(n, d, xt):
    # returns phi(xt), dimension n^d
    phi_xt = np.array([], dtype=float)
    for k in range(1, d + 1):
        combos_k = list(itertools.combinations_with_replacement(np.arange(len(xt)),k))
        # print(combos)
        phi_xt_k = np.ones(len(combos_k), dtype=float)
        for i in range(len(combos_k)):
            tup = combos_k[i]
            for j in tup:
                phi_xt_k[i] *= xt[j]
                
        phi_xt = np.concatenate((phi_xt, phi_xt_k))
    return phi_xt
    

In [90]:
n = 20
d = 3
t = 1000

In [85]:
xt = np.ones(20)
print(len(xt), len(kernelize(4, 3, xt)))

20 1770


In [91]:
phi = np.array([])
for i in range(t):
    # sample haar random from dimension d; then boost it up to the kernel dimension (1000); then compute the effective dimension
    xt = np.random.normal(size=(n, 1))
    xt = xt/np.linalg.norm(xt)
    phi_xt = kernelize(xt, d, xt)
    
    if len(phi) == 0:
        phi = np.array([phi_xt])
    else:
        phi = np.concatenate((phi, np.array([phi_xt])), axis=0)
    # print(xt, np.linalg.norm(xt))
    
print(phi.shape, phi)
    
    

(1000, 1770) [[ 3.09339263e-01  3.84483489e-02  2.70468079e-01 ...  1.05866395e-02
  -6.42105587e-03  3.89452749e-03]
 [-3.47216306e-02 -1.12022031e-01 -3.32621216e-01 ...  1.99374265e-04
   1.10633746e-04  6.13912021e-05]
 [-1.87072547e-01  9.06718943e-02 -1.18096161e-01 ...  3.37743807e-03
   1.22655145e-02  4.45434803e-02]
 ...
 [-2.55269217e-01 -2.09438403e-01  1.06228161e-01 ... -1.40961755e-05
   1.38509400e-06 -1.36099710e-07]
 [ 7.86439813e-02 -2.18018165e-01  3.83598270e-02 ... -2.47108177e-05
  -8.11060792e-04 -2.66207139e-02]
 [-3.47763794e-02 -9.08363229e-02 -2.60402174e-02 ... -5.63050172e-03
   1.25011807e-02 -2.77558780e-02]]


In [92]:
phi.shape

(1000, 1770)

In [97]:
lamb = 0.01

cov = phi @ phi.T
d_eff = np.trace(cov @ (np.linalg.inv(cov + lamb * np.identity(t)))  )
d_eff

922.8795811673052

In [98]:
# Method 2
C_t = phi.T @ phi + lamb * np.identity(len(phi.T))
eig = -np.sort(-np.linalg.eigvals(C_t).real)
d_eff_2 = 0

upper_tail = eig[-1] - lamb
for j in range(1, len(phi.T)):
    idx = len(phi.T) - 1 - j
    if idx * lamb * np.log(t) < upper_tail:
        d_eff_2 = idx + 1
        break
    upper_tail += eig[idx] - lamb
        
    
print(d_eff_2)
    

547
